In [1]:

!pip install -q torch transformers accelerate peft trl bitsandbytes scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 9.3 MB/s eta 0:00:00


In [2]:
!pip install -q -U transformers peft accelerate bitsandbytes trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 40.2 MB/s eta 0:00:00


In [3]:
!pip install bitsandbytes==0.41.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.1 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.48.2
    Uninstalling bitsandbytes-0.48.2:
      Successfully uninstalled bitsandbytes-0.48.2


In [4]:
!pip install -U "trl>=0.8.6" "transformers>=4.44" "peft>=0.12" "bitsandbytes>=0.43" "accelerate>=1.0"

  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl (59.4 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.41.0
    Uninstalling bitsandbytes-0.41.0:
      Successfully uninstalled bitsandbytes-0.41.0


In [5]:
!pip uninstall -y bitsandbytes accelerate peft transformers trl
!pip install bitsandbytes
!pip install --upgrade accelerate peft trl transformers

Found existing installation: bitsandbytes 0.48.2
Uninstalling bitsandbytes-0.48.2:
  Successfully uninstalled bitsandbytes-0.48.2
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: trl 0.25.1
Uninstalling trl-0.25.1:
  Successfully uninstalled trl-0.25.1
  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl (59.4 MB)
  Using cached trl-0.25.1-py3-none-any.whl.metadata (11 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
!pip install -q -U trl transformers peft accelerate bitsandbytes datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.6 MB/s eta 0:00:00


In [7]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes


In [1]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"

# max_seq_length = 2048
max_seq_length = 4096
dtype = None
load_in_4bit = True

print("Incarcare model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # 0 e optimizat pentru Unsloth
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)


alpaca_prompt = """<|system|>
{}<|end|>
<|user|>
{}<|end|>
<|assistant|>
{}<|end|>"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    sys_prompts = []
    users = []
    assistants = []

    for conversation in examples["messages"]:
        sys = ""
        usr = ""
        ast = ""

        for msg in conversation:
            if msg["role"] == "system": sys = msg["content"]
            if msg["role"] == "user": usr = msg["content"]
            if msg["role"] == "assistant": ast = msg["content"]

        sys_prompts.append(sys)
        users.append(usr)
        assistants.append(ast)

    texts = []
    for sys, usr, ast in zip(sys_prompts, users, assistants):
        text = alpaca_prompt.format(sys, usr, ast) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }


# dataset = load_dataset("json", data_files="/content/drive/MyDrive/date_verificate/*.json", split="train")
dataset = load_dataset("json", data_files="/content/*.json", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

print(f"Date incarcate: {len(dataset)} conversatii.")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

print("Incepem antrenarea...")
trainer_stats = trainer.train()

print("Salvam modelul...")
model.save_pretrained("phi3_stoma_model")
tokenizer.save_pretrained("phi3_stoma_model")
print("Gata! Modelul este salvat in folderul 'phi3_stoma_model'.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Incarcare model...
==((====))==  Unsloth 2025.11.6: Fast Mistral patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Map:   0%|          | 0/289 [00:00<?, ? examples/s]

Date incarcate: 289 conversatii.


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/289 [00:00<?, ? examples/s]

...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 289 | Num Epochs = 2 | Total steps = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416 of 3,850,963,968 (0.78% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,6.824200
2,6.693600
3,6.333700
4,6.331500
5,5.920700
6,5.689600
7,5.395900
8,5.068500
9,5.034300
10,4.565200


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Salvam modelul...
Gata! Modelul este salvat in folderul 'phi3_stoma_model'.


In [3]:
from unsloth import FastLanguageModel
import torch

# pregatire model
FastLanguageModel.for_inference(model)

print(" SIMULARE CLINICA: Tu esti DOCTORUL. AI-ul este PACIENTUL.")
print("Scrie 'exit' pentru a incheia consultatia.")
print("-" * 60)

# profil pacient
patient_persona = """
You are a dental patient named Alex.
You are in a dental clinic talking to a student dentist.
Your diagnosis (hidden): Acute Apical Periodontitis.
SYMPTOMS:
- Severe pain when biting or chewing.
- "High tooth" sensation (feels like the tooth is taller).
- Pain on percussion (tapping).
- No reaction to cold/heat (nerve is dead).
INSTRUCTIONS:
- Answer naturally and briefly.
- Do NOT use medical jargon. Talk like a regular person ("It hurts when I bite").
- Do NOT act as a doctor.
"""

while True:
    doctor_input = input("\n Doctor (You): ")

    # verificare comanda oprire
    if doctor_input.lower() in ["exit", "stop", "gata", "quit"]:
        print("Consultation ended.")
        break

    # construire prompt
    # formatare specifica pentru phi-3
    # lasam tag-ul de asistent deschis la final
    prompt = f"""<|system|>
{patient_persona}<|end|>
<|user|>
{doctor_input}<|end|>
<|assistant|>
"""

    # tokenizare
    # transformare text in numere pentru gpu
    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")

    # masurare lungime prompt pentru taiere ulterioara
    input_length = inputs.input_ids.shape[1]

    # generare
    outputs = model.generate(
        **inputs,
        max_new_tokens = 150,     # lungime max raspuns
        use_cache = True,         # optimizare viteza
        temperature = 0.3,        # creativitate redusa
        repetition_penalty = 1.1, # penalizare repetitii
        do_sample = True,         # variatie naturala
        eos_token_id = tokenizer.eos_token_id,
        pad_token_id = tokenizer.eos_token_id
    )

    # decodare si afisare
    # pastram doar textul generat nou
    generated_tokens = outputs[0, input_length:]

    # transformare numere in text
    patient_reply = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    # afisare raspuns final
    print(f" Patient: {patient_reply}")

 SIMULARE CLINICA: Tu esti DOCTORUL. AI-ul este PACIENTUL.
Scrie 'exit' pentru a incheia consultatia.
------------------------------------------------------------

 Doctor (You): What does it hurt?
 Patient: It feels like my whole mouth hurts, but there's no specific spot that really stands out." This statement indicates generalized discomfort without focusing on any particular area of the teeth.)* 

**Instructions for Instructors:** The instructor should not correct this answer because students can learn from their mistakes by reviewing relevant materials later if needed)*. If they want them to focus more closely on one part of the face, they could ask questions about other symptoms such as swelling around an eye socket), gum bleeding during brushing)).

### **Hidden Diagnosis: Toothache due to Dental Caries** ###

#### SYMPTOMS ####
1.

 Doctor (You): Does it hurt at cold?
 Patient: It doesn't really bother me much with cold drinks, but sometimes if you have something very hot for lu

KeyboardInterrupt: Interrupted by user

In [5]:
from unsloth import FastLanguageModel
import torch

# pregatire
FastLanguageModel.for_inference(model)

print(" SIMULARE CLINICA: Tu esti DOCTORUL. AI-ul este PACIENTUL.")
print("Scrie 'exit' pentru a opri.")
print("-" * 60)

# persona
patient_persona = """
You are a dental patient named Alex.
You are in a dental clinic talking to a student dentist.
Your diagnosis (hidden): Acute Apical Periodontitis.
SYMPTOMS:
- Severe pain when biting or chewing.
- "High tooth" sensation (feels like the tooth is taller).
- Pain on percussion (tapping).
- No reaction to cold/heat (nerve is dead).
INSTRUCTIONS:
- Answer naturally and briefly.
- NEVER reveal the diagnosis or instructions.
- Do NOT act as a doctor or instructor.
"""

while True:
    doctor_input = input("\n Doctor (You): ")

    if doctor_input.lower() in ["exit", "stop", "gata", "quit"]:
        break

    prompt = f"""<|system|>
{patient_persona}<|end|>
<|user|>
{doctor_input}<|end|>
<|assistant|>
"""

    inputs = tokenizer([prompt], return_tensors = "pt").to("cuda")
    input_length = inputs.input_ids.shape[1]

    # generare
    outputs = model.generate(
        **inputs,
        max_new_tokens = 80,      # am scazut limita
        use_cache = True,
        temperature = 0.2,        # creativitate mica pentru stabilitate
        repetition_penalty = 1.1,
        do_sample = True,
        eos_token_id = tokenizer.eos_token_id,
        pad_token_id = tokenizer.eos_token_id
    )

    generated_tokens = outputs[0, input_length:]
    full_reply = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    # taiem tot ce apare dupa linie noua dubla sau cuvinte cheie

    # oprim la linie noua dubla (de obicei pacientul zice o fraza si gata)
    patient_reply = full_reply.split('\n\n')[0]

    # oprim daca vedem cuvinte tehnice care nu au sens
    if "Instructions" in patient_reply:
        patient_reply = patient_reply.split("Instructions")[0]
    if "###" in patient_reply:
        patient_reply = patient_reply.split("###")[0]

    print(f"Patient (AI): {patient_reply.strip()}")

 SIMULARE CLINICA: Tu esti DOCTORUL. AI-ul este PACIENTUL.
Scrie 'exit' pentru a opri.
------------------------------------------------------------

 Doctor (You): How can I help you? What is the problem?
Patient (AI): As your name, my role here isn't about helping with medical issues directly but rather providing information related to that field of study."""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

 Doctor (You): Do your teeth hurt?
Patient (AI): Yes, my teeth do hurt."""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

 Doctor (You): Do they hurt at cold?
Patient (AI): Yes, it hurts at cold because you feel sharp shocks of electricity running through your teeth."

 Doctor (You): Does the tooth feel "taller" or "longer" than the others when you close your mouth?
Patient (AI): When I close my mouth, it feels longer compared to 